# Lab | Web Scraping Multiple Pages

In [1]:
from time import sleep
from random import randint
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Part 1

In [2]:
url = "https://open.spotify.com/playlist/1EVE9kOZ2i4171hNdvWVhU"

response = requests.get(url)
response.status_code

200

In [4]:
soup = BeautifulSoup(response.content, "html.parser")
#print(soup.prettify())

In [18]:
titles = []
artists = []

# iterating over each page of results
for page_num in range(1, 4):  # 4 is the number of pages to scrape
# (pagination) in the case of Spotify, when you browse a playlist or album, there is a certain number of songs that may not 
# fit all on one page. So, the data is divided into pages to allow users to gradually browse through it

    # waiting to avoid too rapid requests to the server
    sleep(randint(1, 3))
    
    url = f"https://open.spotify.com/playlist/1EVE9kOZ2i4171hNdvWVhU?page={page_num}"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # getting the links to the songs on this page
    song_urls = [
        tag["content"] 
        for tag in soup.find_all("meta", {"name": "music:song"})
    ]

    # iterating over each link and get title and artist (hier artist is still a next link)
    for song_url in song_urls:
        sleep(randint(1, 3))
        
        # getting title and artist for each link
        response_song = requests.get(song_url)
        soup_song = BeautifulSoup(response_song.content, "html.parser")
        
        # finding title and artist
        title = soup_song.find("meta", {"property": "og:title"})["content"]
        artist_url = soup_song.find("meta", {"name": "music:musician"})["content"]
        
        # getting artist name from link
        response_artist = requests.get(artist_url)
        soup_artist = BeautifulSoup(response_artist.content, "html.parser")
        artist = soup_artist.find("meta", {"property": "og:title"})["content"]
        
        # adding to lists
        titles.append(title)
        artists.append(artist)

        
df = pd.DataFrame({"song_title": titles, "artist": artists})

# saving to a CSV file
#df.to_csv("spotify_songs.csv", index=False)

display(df)

,song_title,artist
0,Flowers,Miley Cyrus
1,Boy's a liar Pt. 2,PinkPantheress
2,Miss You,Oliver Tree
3,Seven (feat. Latto) (Explicit Ver.),Jung Kook
4,Daylight,David Kushner
...,...,...
85,Eyes Closed,Ed Sheeran
86,Don't You Worry,Black Eyed Peas
87,Days Of Summer,Smiley
88,Past Lives,Lee Isaacs
